In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import sys
import pickle

In [30]:
data = pd.read_csv('Robot_train_4s.csv')

# 1. Data Preprocessing
# Example: Handle missing values
data = data.dropna()
data

,Label,X,Y,Z,X*Y,X*Z,Y*Z,X*Y+Z,X*Y*Z,X*Z+Y,...,slop(1/z),slop(1/x),slop(z/x),slop(z),slop(x)^2,slop(x/z)^2,slop(1/z)^2,slop(1/x)^2,slop(z/x)^2,slop(z)^2
0,-1,1044.181085,0.746138,0.387314,779.103186,404.425953,404.425953,779.490500,301.757571,405.172091,...,91.199270,0.62300,0.006832,0.010965,2.575925,21424.762920,8320.000000,0.388000,0.000047,0.000120
1,-1,1042.576116,0.746192,0.376349,777.961957,392.372479,392.372479,778.338306,292.785205,393.118671,...,153.092468,0.62300,0.004067,0.006532,2.579364,60453.348740,23400.000000,0.388000,0.000017,0.000043
2,-1,1040.970076,0.746208,0.369817,776.780198,384.968431,384.968431,777.150015,287.266523,385.714639,...,194.628260,0.62300,0.003198,0.005138,2.580556,97751.884530,37900.000000,0.388000,0.000010,0.000026
3,-1,1039.363665,0.746214,0.364679,775.587718,379.034102,379.034102,775.952397,282.840553,379.780316,...,212.856535,0.62200,0.002924,0.004698,2.580942,116937.066700,45300.000000,0.387000,0.000009,0.000022
4,-1,1037.757134,0.746215,0.359981,774.389940,373.572851,373.572851,774.749921,278.765665,374.319066,...,219.394471,0.62200,0.002837,0.004558,2.581067,124236.916600,48100.000000,0.387000,0.000008,0.000021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145678,16&17,-484.315636,0.696923,-121.823592,-337.530706,59001.070440,-84.901663,-459.354298,41119.203010,59001.767360,...,7.697518,0.61500,0.079893,0.129912,2.644100,156.667677,59.300000,0.378000,0.006380,0.016877
145679,16&17,-485.941705,0.696923,-121.953504,-338.663951,59262.293660,-84.992202,-460.617455,41301.255490,59262.990590,...,7.697459,0.61500,0.079894,0.129913,2.644097,156.665072,59.300000,0.378000,0.006380,0.016877
145680,16&17,-487.567773,0.696923,-122.083417,-339.797195,59523.939750,-85.082741,-461.880612,41483.602660,59524.636670,...,7.697518,0.61500,0.079893,0.129912,2.644100,156.667677,59.300000,0.378000,0.006380,0.016877
145681,16&17,-489.193842,0.696923,-122.213329,-340.930440,59786.007960,-85.173280,-463.143769,41666.244020,59786.704880,...,7.697459,0.61500,0.079894,0.129913,2.644100,156.665265,59.300000,0.378000,0.006380,0.016877


In [31]:
print(len(set(data['Label'])))

154


In [32]:
# Split the labels into two separate integer columns
def split_labels(label):
    if label == '-1':
        return -1, -1
    else:
        return map(int, label.split('&' if '&' in label else ','))

data[['label1', 'label2']] = data['Label'].apply(split_labels).apply(pd.Series)
data = data.drop(columns=['Label'])

In [33]:
print(set(data['label1']))
print(set(data['label2']))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, -1}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, -1}


In [34]:
data

,X,Y,Z,X*Y,X*Z,Y*Z,X*Y+Z,X*Y*Z,X*Z+Y,Y*Z+X,...,slop(z/x),slop(z),slop(x)^2,slop(x/z)^2,slop(1/z)^2,slop(1/x)^2,slop(z/x)^2,slop(z)^2,label1,label2
0,1044.181085,0.746138,0.387314,779.103186,404.425953,404.425953,779.490500,301.757571,405.172091,1044.470075,...,0.006832,0.010965,2.575925,21424.762920,8320.000000,0.388000,0.000047,0.000120,-1,-1
1,1042.576116,0.746192,0.376349,777.961957,392.372479,392.372479,778.338306,292.785205,393.118671,1042.856945,...,0.004067,0.006532,2.579364,60453.348740,23400.000000,0.388000,0.000017,0.000043,-1,-1
2,1040.970076,0.746208,0.369817,776.780198,384.968431,384.968431,777.150015,287.266523,385.714639,1041.246036,...,0.003198,0.005138,2.580556,97751.884530,37900.000000,0.388000,0.000010,0.000026,-1,-1
3,1039.363665,0.746214,0.364679,775.587718,379.034102,379.034102,775.952397,282.840553,379.780316,1039.635794,...,0.002924,0.004698,2.580942,116937.066700,45300.000000,0.387000,0.000009,0.000022,-1,-1
4,1037.757134,0.746215,0.359981,774.389940,373.572851,373.572851,774.749921,278.765665,374.319066,1038.025757,...,0.002837,0.004558,2.581067,124236.916600,48100.000000,0.387000,0.000008,0.000021,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145678,-484.315636,0.696923,-121.823592,-337.530706,59001.070440,-84.901663,-459.354298,41119.203010,59001.767360,-569.217299,...,0.079893,0.129912,2.644100,156.667677,59.300000,0.378000,0.006380,0.016877,16,17
145679,-485.941705,0.696923,-121.953504,-338.663951,59262.293660,-84.992202,-460.617455,41301.255490,59262.990590,-570.933907,...,0.079894,0.129913,2.644097,156.665072,59.300000,0.378000,0.006380,0.016877,16,17
145680,-487.567773,0.696923,-122.083417,-339.797195,59523.939750,-85.082741,-461.880612,41483.602660,59524.636670,-572.650514,...,0.079893,0.129912,2.644100,156.667677,59.300000,0.378000,0.006380,0.016877,16,17
145681,-489.193842,0.696923,-122.213329,-340.930440,59786.007960,-85.173280,-463.143769,41666.244020,59786.704880,-574.367122,...,0.079894,0.129913,2.644100,156.665265,59.300000,0.378000,0.006380,0.016877,16,17


In [35]:
# Features and targets
Y = data[['label1', 'label2']]
data.pop('label1')
data.pop('label2')
X = data

In [36]:
print(X.shape,Y.shape)

(145683, 38) (145683, 2)


In [37]:
print(Y)

        label1  label2
0           -1      -1
1           -1      -1
2           -1      -1
3           -1      -1
4           -1      -1
...        ...     ...
145678      16      17
145679      16      17
145680      16      17
145681      16      17
145682      16      17

[145683 rows x 2 columns]


In [46]:
# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the base classifier
rf = RandomForestClassifier(n_estimators=11, random_state=42,max_depth=10)

In [47]:
# Wrap it in a MultiOutputClassifier
multi_target_rf = MultiOutputClassifier(rf, n_jobs=-1)

# Train the model
multi_target_rf.fit(X_train, Y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=10,
                                                       n_estimators=11,
                                                       random_state=42),
                      n_jobs=-1)

In [48]:
# Make predictions
Y_pred = multi_target_rf.predict(X_test)

In [49]:
print(Y_pred)

[[ 7 14]
 [ 9 17]
 [ 3 10]
 ...
 [ 2 15]
 [ 3  4]
 [ 6 16]]


In [50]:
# Convert predictions to DataFrame for easy evaluation
Y_pred_df = pd.DataFrame(Y_pred, columns=Y_test.columns, index=Y_test.index)

Y_pred_df

,label1,label2
99823,7,14
118842,9,17
52131,3,10
80087,5,14
7969,0,8
...,...,...
126099,11,12
11816,0,12
44081,2,15
46373,3,4


In [51]:
# Classification report for each target
for column in Y_test.columns:
    print(f"Classification report for {column}:")
    print(classification_report(Y_test[column], Y_pred_df[column]))

Classification report for label1:
              precision    recall  f1-score   support

          -1       1.00      0.99      1.00       198
           0       0.98      1.00      0.99      3149
           1       0.99      0.98      0.98      3056
           2       0.99      0.94      0.96      2886
           3       0.94      1.00      0.97      2598
           4       0.99      0.97      0.98      2420
           5       0.97      0.99      0.98      2290
           6       0.99      0.99      0.99      2048
           7       0.99      0.99      0.99      1868
           8       0.99      0.97      0.98      1769
           9       0.97      0.88      0.93      1509
          10       1.00      0.98      0.99      1352
          11       1.00      0.98      0.99      1161
          12       0.76      0.91      0.83       958
          13       0.99      0.97      0.98       771
          14       1.00      1.00      1.00       526
          15       1.00      0.99      0.99    

In [52]:
# Calculate and print the accuracy score for each target
for column in Y_test.columns:
    accuracy = accuracy_score(Y_test[column], Y_pred_df[column])
    print(f"Accuracy for {column}: {accuracy:.2f}")

Accuracy for label1: 0.97
Accuracy for label2: 0.97


In [53]:
# Serialize the model to a bytes object
model_bytes = pickle.dumps(multi_target_rf)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000000, " M bytes")

Memory usage of Random Forest model: 2.614341  M bytes
